In [34]:
import os
import openai
import requests
import json
import sseclient
import pandas as pd
import json

import dotenv
from os import environ
env_file = '.env'

In [36]:
dotenv.load_dotenv(env_file, override=True)
openai.api_key = os.getenv("OPENAI_API_KEY")

In [5]:
docs_chunks_path = "create_chunks/docs_chunks.json"
with open(docs_chunks_path, 'r', encoding="utf8") as f:
    docs_chunks = json.load(f)

len(docs_chunks), docs_chunks[:2]

(338,
 [{'header': ['Alert'],
   'link': '/docs/alert',
   'content': "A small box to quickly grab the user's attention and communicate a brief message. An alert is an element that notifies the user of something important that is not user initiated. If you need to alert the user because of something they did, consider using a toast notification or an inline message. Instead, an alert is used to notify the user of something unrelated to what they just did. Examples of this might be announcing a new feature within the product, or alerting the user of scheduled server maintenance and downtime."},
  {'header': ['Alert', 'Appearance'],
   'link': '/docs/alert#appearance',
   'content': 'The alert message must be concise, we recommend no more than 256 characters. If you feel the need to explain in further detail, you may use an optional hyperlink at the end of the alert message to explain more on a new page. You may also include an optional close icon (close icon is required for sticky alert

In [39]:
def get_embeddings(input_content, model="text-embedding-ada-002"):
    response = openai.Embedding.create(
        model=model,
        input=f"{input_content}"
    )
    return response

In [7]:
embeddings_df = pd.read_parquet("data/embeddings_df.parquet", engine='fastparquet')
embeddings_df.head()

,chunk,object,embedding_object,index,embedding,prompt_tokens,total_tokens
level_0,,,,,,,
0,"{'header': ['Alert'], 'link': '/docs/alert', '...",list,embedding,0,"[-0.0216156505048275, 0.03179503604769707, 0.0...",122,122
1,"{'header': ['Alert', 'Appearance'], 'link': '/...",list,embedding,0,"[-0.002292848890647292, 0.04747709631919861, 0...",88,88


In [23]:
# Check if `chunk` is already in `embeddings_df`
def check_if_chunk_in_embeddings_df(chunk):
  return embeddings_df[embeddings_df["chunk"] == chunk].shape[0] > 0

In [25]:
print(check_if_chunk_in_embeddings_df(docs_chunks[0]))
print(check_if_chunk_in_embeddings_df(docs_chunks[1]))
print(check_if_chunk_in_embeddings_df(embeddings_df.loc[0, "chunk"]))
print(check_if_chunk_in_embeddings_df(embeddings_df.loc[1, "chunk"]))

False
True
True
True


In [37]:
remaining_chunks = [chunk for chunk in docs_chunks if not check_if_chunk_in_embeddings_df(chunk)]

# Safety check: Confirm code works before running on all chunks
remaining_chunks = remaining_chunks[:1]

f"{len(remaining_chunks)}/{len(docs_chunks)} chunks remaining to be embedded"

'1/338 chunks remaining to be embedded'

In [40]:
response_cache = None

for chunk in remaining_chunks:
    response = get_embeddings(chunk)
    response_cache = response

    embeddings_df.loc[len(embeddings_df)] = {
        "chunk": chunk,
        "object": response["object"],
        "embedding_object": response["data"][0]["object"],
        "index": response["data"][0]["index"],
        "embedding": response["data"][0]["embedding"],
        "prompt_tokens": response["usage"]["prompt_tokens"],
        "total_tokens": response["usage"]["total_tokens"],
    }

    # Safety check: Confirm code works before running on all chunks
    break

In [41]:
response_cache

<OpenAIObject list at 0x216ce8e5540> JSON: {
  "data": [
    {
      "embedding": [
        -0.022394228726625443,
        0.03163980692625046,
        0.00710719171911478,
        3.4493583370931447e-05,
        0.01846347376704216,
        -0.00559163698926568,
        -0.009480869397521019,
        -0.01612439751625061,
        -0.014082897454500198,
        -0.009127931669354439,
        -0.008345933631062508,
        0.008082959800958633,
        -0.03399272263050079,
        -0.0035778172314167023,
        -0.008941082283854485,
        0.006069140508770943,
        0.01876796968281269,
        -0.021093204617500305,
        0.0009697130299173295,
        -0.0102144256234169,
        -0.011127910576760769,
        0.010636566206812859,
        0.002756026340648532,
        0.020055152475833893,
        -0.011916830204427242,
        0.003854630747810006,
        0.017688395455479622,
        -0.028927031904459,
        -0.0032802424393594265,
        -0.02477482706308365,
       

In [51]:
embeddings_df.loc[2, :]["chunk"] == remaining_chunks[0]

True